# Analisi dei dati raccolti

## Setup

In [1]:
import pandas as pd

In [2]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconhashv12

In [3]:
path = "../asconhashv12/data/{}.csv"

### Armv6m

In [4]:
with open(path.format("armv6m"), "r") as f:
    ah_armv6m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [5]:
# Generazione hash
data = ah_armv6m["0B"]

print_data(data)

0B
235    468
234    173
242    169
237    114
236     55
241     13
243      8
Name: count, dtype: int64

Massimo: 243
Minimo: 234
Media: 236.435
Varianza: 7.893668668668668


#### PT 8 byte

In [6]:
# Generazione hash
data = ah_armv6m["8B"]

print_data(data)

8B
290    265
288    240
296    150
289    149
295    124
291     70
297      2
Name: count, dtype: int64

Massimo: 297
Minimo: 288
Media: 290.975
Varianza: 8.83521021021021


#### PT 16 byte

In [7]:
# Generazione hash
data = ah_armv6m["16B"]

print_data(data)

16B
342    333
344    319
349    205
343     63
350     58
341     21
345      1
Name: count, dtype: int64

Massimo: 350
Minimo: 341
Media: 344.582
Varianza: 8.467743743743744


#### PT 32 byte

In [8]:
# Generazione hash
data = ah_armv6m["32B"]

print_data(data)

32B
451    395
456    224
449    165
457    129
450     40
458     20
452     15
459     12
Name: count, dtype: int64

Massimo: 459
Minimo: 449
Media: 452.775
Varianza: 9.56594094094094


#### PT 64 byte

In [9]:
# Generazione hash
data = ah_armv6m["64B"]

print_data(data)

64B
671    240
665    210
673    178
666    176
672    119
670     72
674      5
Name: count, dtype: int64

Massimo: 674
Minimo: 665
Media: 669.278
Varianza: 9.95266866866867


#### PT 128 byte

In [10]:
# Generazione hash
data = ah_armv6m["128B"]

print_data(data)

128B
1101    405
1102    390
1109     69
1103     59
1104     44
1108     24
1110      6
1105      3
Name: count, dtype: int64

Massimo: 1110
Minimo: 1101
Media: 1102.426
Varianza: 5.119643643643644


#### PT 256 byte

In [11]:
# Generazione hash
data = ah_armv6m["256B"]

print_data(data)

256B
1968    670
1969    285
1966     23
1967     16
1961      5
1970      1
Name: count, dtype: int64

Massimo: 1970
Minimo: 1961
Media: 1968.19
Varianza: 0.6065065065065065


#### PT 512 byte

In [12]:
# Generazione hash
data = ah_armv6m["512B"]

print_data(data)

512B
3702    292
3695    190
3700    177
3701    145
3694    116
3703     37
3699     27
3696     16
Name: count, dtype: int64

Massimo: 3703
Minimo: 3694
Media: 3699.103
Varianza: 10.060451451451453


#### PT 1024 byte

In [13]:
# Generazione hash
data = ah_armv6m["1024B"]

print_data(data)

1024B
7160    480
7159    161
7161    125
7162    125
7167     64
7168     24
7163     11
7166     10
Name: count, dtype: int64

Massimo: 7168
Minimo: 7159
Media: 7160.947
Varianza: 5.025216216216215


### Armv6m_lowsize

In [14]:
with open(path.format("armv6m_lowsize"), "r") as f:
    ah_lowsize = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [15]:
# Generazione hash
data = ah_lowsize["0B"]

print_data(data)

0B
238    593
237    366
247     18
240      8
239      8
246      7
Name: count, dtype: int64

Massimo: 247
Minimo: 237
Media: 237.876
Varianza: 2.298922922922923


#### PT 8 byte

In [16]:
# Generazione hash
data = ah_lowsize["8B"]

print_data(data)

8B
301    366
292    247
302    173
293    105
300     55
299     54
Name: count, dtype: int64

Massimo: 302
Minimo: 292
Media: 297.947
Varianza: 17.868059059059064


#### PT 16 byte

In [17]:
# Generazione hash
data = ah_lowsize["16B"]

print_data(data)

16B
356    428
347    317
357    121
349     81
348     27
354     13
355     12
358      1
Name: count, dtype: int64

Massimo: 358
Minimo: 347
Media: 352.449
Varianza: 18.97237137137137


#### PT 32 byte

In [18]:
# Generazione hash
data = ah_lowsize["32B"]

print_data(data)

32B
457    449
456    362
466     88
465     52
458     15
459     15
464      9
467      9
463      1
Name: count, dtype: int64

Massimo: 467
Minimo: 456
Media: 458.05
Varianza: 11.178678678678677


#### PT 64 byte

In [19]:
# Generazione hash
data = ah_lowsize["64B"]

print_data(data)

64B
675    301
676    221
685    218
684    186
677     48
678     24
679      2
Name: count, dtype: int64

Massimo: 685
Minimo: 675
Media: 679.251
Varianza: 19.47547447447447


#### PT 128 byte

In [20]:
# Generazione hash
data = ah_lowsize["128B"]

print_data(data)

128B
1122    671
1123    286
1131     15
1132      9
1129      9
1130      5
1124      4
1125      1
Name: count, dtype: int64

Massimo: 1132
Minimo: 1122
Media: 1122.625
Varianza: 2.799174174174174


#### PT 256 byte

In [21]:
# Generazione hash
data = ah_lowsize["256B"]

print_data(data)

256B
2007    712
2008    222
2006     54
2009      8
2010      4
Name: count, dtype: int64

Massimo: 2010
Minimo: 2006
Media: 2007.196
Varianza: 0.30588988988988997


#### PT 512 byte

In [22]:
# Generazione hash
data = ah_lowsize["512B"]

print_data(data)

512B
3777    536
3776    351
3775     64
3778     49
Name: count, dtype: int64

Massimo: 3778
Minimo: 3775
Media: 3776.57
Varianza: 0.47157157157157154


#### PT 1024 byte

In [23]:
# Generazione hash
data = ah_lowsize["1024B"]

print_data(data)

1024B
7307    484
7306    217
7308     86
7305     73
7304     54
7309     32
7316     28
7314      9
7315      8
7317      6
7313      2
7310      1
Name: count, dtype: int64

Massimo: 7317
Minimo: 7304
Media: 7307.079
Varianza: 5.10986886886887


### Bi32_armv6m

In [24]:
with open(path.format("bi32_armv6m"), "r") as f:
    ah_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [25]:
# Generazione hash
data = ah_bi32["0B"]

print_data(data)

0B
240    386
239    183
247    162
242    136
241     81
249     21
246     16
248     14
250      1
Name: count, dtype: int64

Massimo: 250
Minimo: 239
Media: 241.711
Varianza: 9.100579579579579


#### PT 8 byte

In [26]:
# Generazione hash
data = ah_bi32["8B"]

print_data(data)

8B
294    410
301    174
296    154
303     98
293     85
295     45
302     28
304      6
Name: count, dtype: int64

Massimo: 304
Minimo: 293
Media: 296.652
Varianza: 12.581477477477474


#### PT 16 byte

In [27]:
# Generazione hash
data = ah_bi32["16B"]

print_data(data)

16B
348    417
350    220
355    157
357    116
358     41
356     32
349     17
Name: count, dtype: int64

Massimo: 358
Minimo: 348
Media: 351.266
Varianza: 13.480724724724729


#### PT 32 byte

In [28]:
# Generazione hash
data = ah_bi32["32B"]

print_data(data)

32B
456    244
458    155
457    122
465    110
466    106
464    106
463     82
459     69
467      4
460      2
Name: count, dtype: int64

Massimo: 467
Minimo: 456
Media: 460.163
Varianza: 14.875306306306305


#### PT 64 byte

In [29]:
# Generazione hash
data = ah_bi32["64B"]

print_data(data)

64B
682    371
675    194
680    136
673    130
683     96
681     51
674     19
676      3
Name: count, dtype: int64

Massimo: 683
Minimo: 673
Media: 679.075
Varianza: 13.508883883883886


#### PT 128 byte

In [30]:
# Generazione hash
data = ah_bi32["128B"]

print_data(data)

128B
1116    416
1115    371
1123     69
1117     64
1122     38
1118     30
1124     10
1119      2
Name: count, dtype: int64

Massimo: 1124
Minimo: 1115
Media: 1116.55
Varianza: 5.665165165165167


#### PT 256 byte

In [31]:
# Generazione hash
data = ah_bi32["256B"]

print_data(data)

256B
1991    621
1992    343
1984     17
1993     10
1985      5
1989      2
1990      2
Name: count, dtype: int64

Massimo: 1993
Minimo: 1984
Media: 1991.208
Varianza: 1.3641001001001003


#### PT 512 byte

In [32]:
# Generazione hash
data = ah_bi32["512B"]

print_data(data)

512B
3743    423
3741    142
3736    124
3744    113
3742     69
3734     60
3735     46
3737     13
3733     10
Name: count, dtype: int64

Massimo: 3744
Minimo: 3733
Media: 3740.806
Varianza: 11.295659659659663


#### PT 1024 byte

In [33]:
# Generazione hash
data = ah_bi32["1024B"]

print_data(data)

1024B
7237    331
7238    256
7239    135
7245    118
7240     87
7244     50
7246     16
7247      7
Name: count, dtype: int64

Massimo: 7247
Minimo: 7237
Media: 7239.295
Varianza: 8.31829329329329


### Ref

In [34]:
with open(path.format("ref"), "r") as f:
    ah_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [35]:
# Generazione hash
data = ah_ref["0B"]

print_data(data)

0B
387    444
396    183
394    111
388     77
397     67
389     60
395     56
390      2
Name: count, dtype: int64

Massimo: 397
Minimo: 387
Media: 390.745
Varianza: 16.872847847847847


#### PT 8 byte

In [36]:
# Generazione hash
data = ah_ref["8B"]

print_data(data)

8B
465    465
474    283
467    156
472     25
464     23
466     20
475     17
473     10
468      1
Name: count, dtype: int64

Massimo: 475
Minimo: 464
Media: 468.284
Varianza: 16.39573973973974


#### PT 16 byte

In [37]:
# Generazione hash
data = ah_ref["16B"]

print_data(data)

16B
552    347
543    271
545    105
550     95
542     60
551     59
544     41
553     22
Name: count, dtype: int64

Massimo: 553
Minimo: 542
Media: 547.671
Varianza: 17.458217217217214


#### PT 32 byte

In [38]:
# Generazione hash
data = ah_ref["32B"]

print_data(data)

32B
708    362
707    203
706    115
700     92
701     73
698     54
699     52
705     34
709     15
Name: count, dtype: int64

Massimo: 709
Minimo: 698
Media: 705.225
Varianza: 12.372747747747749


#### PT 64 byte

In [39]:
# Generazione hash
data = ah_ref["64B"]

print_data(data)

64B
1019    891
1020     58
1021     31
1027      6
1022      6
1026      5
1018      2
1028      1
Name: count, dtype: int64

Massimo: 1028
Minimo: 1018
Media: 1019.228
Varianza: 0.8969129129129126


#### PT 128 byte

In [40]:
# Generazione hash
data = ah_ref["128B"]

print_data(data)

128B
1651    479
1649    136
1642    130
1644    103
1650     53
1643     37
1652     37
1641     22
1645      3
Name: count, dtype: int64

Massimo: 1652
Minimo: 1641
Media: 1648.287
Varianza: 13.540171171171174


#### PT 256 byte

In [41]:
# Generazione hash
data = ah_ref["256B"]

print_data(data)

256B
2906    580
2905    242
2907     76
2904     52
2898     15
2899     14
2903     14
2897      3
2896      3
2900      1
Name: count, dtype: int64

Massimo: 2907
Minimo: 2896
Media: 2905.407
Varianza: 2.5278788788788784


#### PT 512 byte

In [42]:
# Generazione hash
data = ah_ref["512B"]

print_data(data)

512B
5415    283
5416    224
5425    137
5424    116
5417     85
5423     80
5422     45
5418     20
5426      7
5414      3
Name: count, dtype: int64

Massimo: 5426
Minimo: 5414
Media: 5418.897
Varianza: 16.845236236236236


#### PT 1024 byte

In [43]:
# Generazione hash
data = ah_ref["1024B"]

print_data(data)

1024B
10444    253
10453    211
10451    111
10443    110
10446    106
10452     88
10445     68
10450     27
10454     17
10441      3
10447      2
10440      2
10449      2
Name: count, dtype: int64

Massimo: 10454
Minimo: 10440
Media: 10447.881
Varianza: 16.517356356356355
